In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
! pip install emoji
import emoji

# some NLP libraries that can help us with preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')

! pip install tweet_preprocessor
import preprocessor as tweet_preprocessor
! pip install twitter-text-python
from ttp import ttp
! pip install emoji
import emoji
import torch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LoG1c\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LoG1c\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('./data.csv')

In [3]:
def convert_to_emoji(text):
    return emoji.demojize(text)

In [4]:
tweets = data['tweet']
tweets_tokenized = []
tt = nltk.tokenize.TweetTokenizer()
for tweet in tweets:
    tweet = convert_to_emoji(tweet)
    # initializing punctuations string
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~"'?'''
 
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in tweet:
        if ele in punc:
            tweet = tweet.replace(ele, "")
 
    tweets_tokenized.append(tt.tokenize(text=tweet))

In [5]:
#remove stopwords
stemmer = nltk.stem.porter.PorterStemmer()
stoplist = nltk.corpus.stopwords.words('english')

stemmed = []

for tweet in tweets_tokenized:
    processed_tokens = [stemmer.stem(t) for t in tweet if t not in stoplist]
    stemmed.append(processed_tokens)

In [6]:
def super_simple_preprocess(text):
  # lowercase
  text = text.lower()
  # remove non alphanumeric characters
  text = re.sub('[^A-Za-z0-9 ]+','', text)
  return text

In [7]:
data_updated = []
for tweet in stemmed:
    data_updated.append(super_simple_preprocess(" ".join(tweet)))
pd.DataFrame(data_updated).shape

(10600, 1)

In [8]:
data_updated = pd.DataFrame(data_updated)
data_updated.insert(1, 'w', data['label'])
data = data_updated

In [9]:
data.head()

,0,w
0,the cdc current report 99031 death in gener di...,real
1,state report 1121 death small rise last tuesda...,real
2,polit correct woman almost use pandem excus no...,fake
3,indiafightscorona we 1524 covid test laborator...,real
4,popul state gener larg case count look new cas...,real


In [10]:
sentences = data.iloc[:, 0]

In [11]:
labels = data['w']

In [12]:
labels.replace('real',1, inplace=True)
labels.replace('fake',0, inplace=True)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

In [14]:
X = X.toarray()

In [15]:
y = labels

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train : ", len(X_train),"Test : ", len(X_test),"Val : ", len(X_val))

Train :  8480 Test :  1060 Val :  1060


In [24]:
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test.values)
X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val.values)


In [25]:
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [26]:
# Define neural network model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [27]:
# Instantiate model, loss, and optimizer
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [28]:
# Define DataLoader
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [29]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_val)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_val).sum().item() / len(y_val)
    print(f'Accuracy: {accuracy}')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)